# Gradient Descent Using Different Python Libraries

A polynomial expansion of a function $f$ is defined as:

\begin{equation}

f(x) = \sum^{N}_{k=0}{w_k x^k} = w_0 + w_1x + w_2x^2 + w_3x^3 + w_4x^4 + w_5x^5 .....

\end{equation}

In this demonstration, we attempt to calculate the coefficients $w_k$ in the polynomial expansion of $sin(x)$. By gradient descent, the optimal value for each weight $w_k$ can be found when 

\begin{equation}
\frac{\partial L}{\partial w_k} \approx 0
\end{equation}

And the loss function $L$ is the $L_2$ loss of our approximation, defined as

\begin{equation}
L = (\hat{y} - y)^2 \quad \text{$\hat{y}$: true value of $sin(x)$}
\end{equation}

Taking the partial derivative of $L$ with respect to each weight, the gradient can be found
\begin{equation}
\frac{\partial L}{\partial w_k} = \frac{\partial (\hat{y}-y)^2}{\partial w_k} = 2(\hat{y}-y)x^k
\end{equation}

In [ ]:
# Code adapted from https://pytorch.org/tutorials/beginner/pytorch_with_examples.html# 

import numpy as np

N_SAMPLES = 4000
LEARNING_RATE = 1e-6

# Create random input and output data
x = np.linspace(-np.pi, np.pi, N_SAMPLES)
y = np.sin(x)

# Randomly initialize weights
np.random.seed(0)
w = np.random.random(8)

for t in range(4000):

    y_pred = w[0] + w[1]*x + w[2]*(x**2) + w[3]*(x**3) + w[4]*(x**4) + w[5]*(x**5) + w[6]*(x**6) + w[7]*(x**7)

    error = y_pred - y
    loss = (error ** 2).mean()
    gradients = np.array((
        2 * error,
        2 * error * x,
        2 * error * (x**2),
        2 * error * (x**3),
        2 * error * (x**4),
        2 * error * (x**5),
        2 * error * (x**6),
        2 * error * (x**7)
    )).mean(-1)
    w -= gradients * LEARNING_RATE
    
    if t % 200 == 199:
        print(f"Loss at {t+1:> 5} iteration: {loss:> 5.3f}")

In [ ]:
import torch

dtype = torch.float
device = torch.device("cuda")
torch.manual_seed(0)

N_SAMPLES = 4000
LEARNING_RATE = 1e-6

x = torch.linspace(-torch.pi, torch.pi, N_SAMPLES, device=device, dtype=dtype)
y = torch.sin(x)
w = torch.randn(8, device = device, dtype = dtype)

for t in range(4000):
    y_pred = w[0] + w[1]*x + w[2]*x**2 + w[3]*x**3 + w[4]*x**4 + w[5]*x**5 + w[6]*x**6 + w[7]*x**7

    error = y_pred - y
    loss = error.pow(2).mean().item()

    gradients = torch.empty_like(w, device = device)
    gradients[0] = (2 * error).mean(-1)
    gradients[1] = (2 * error * x).mean(-1)
    gradients[2] = (2 * error * (x**2)).mean(-1)
    gradients[3] = (2 * error * (x**3)).mean(-1)
    gradients[4] = (2 * error * (x**4)).mean(-1)
    gradients[5] = (2 * error * (x**5)).mean(-1)
    gradients[6] = (2 * error * (x**6)).mean(-1)
    gradients[7] = (2 * error * (x**7)).mean(-1)

    """ 
    ValueError: only tensor with size 1 can be converted to python scalar
    gradients = torch.tensor((
        2 * error,
        2 * error * x,
        2 * error * (x**2),
        2 * error * (x**3),
        2 * error * (x**4),
        2 * error * (x**5),
        2 * error * (x**6),
        2 * error * (x**7)), device = device).mean(-1)
    gradients = torch.from_numpy(__gradients, device = device)
    """

    w -= gradients * LEARNING_RATE
    
    if t % 200 == 199:
        print(f"Loss at {t+1:> 5} iteration: {loss:> 5.3f}")


In [27]:
import torch

dtype = torch.float
device = "cuda"
torch.set_default_device(device)
torch.manual_seed(0)

N_SAMPLES = 4000
LEARNING_RATE = 1e-6

x = torch.linspace(-torch.pi, torch.pi, N_SAMPLES, dtype=dtype)
y = torch.sin(x)
w = torch.randn(8, device = device, dtype = dtype, requires_grad = True)

for t in range(4000):
    y_pred = w[0] + w[1]*x + w[2]*x**2 + w[3]*x** 3 + w[4]*x**4 + w[5]*x**5 + w[6]*x**6 + w[7]*x**7

    error = y_pred - y
    loss = error.pow(2).mean()

    loss.backward()

    with torch.no_grad():
        w -= w.grad * LEARNING_RATE
        w.grad[:] = 0
        
    if t % 200 == 199:
        print(f"Loss at {t+1:> 5} iteration: {loss:> 5.3f}")


ValueError: only one element tensors can be converted to Python scalars

## Custom autograd functions
### Using Legendre polynomial approximation of sine function as an example.

ref:

[Legendre polynomial -- Academic Accelerator](https://academic-accelerator.com/encyclopedia/legendre-polynomials)

[Legendre polynomial -- Wolfram Mathworld](https://mathworld.wolfram.com/LegendrePolynomial.html)


An arbritary function, $f(x)$, in domain $x \in [-1, 1]$ can be approximated as such:
\begin{equation}
f(x) = \sum^{\infty}_{n=0}{a_n}{P_n} = w_0P_0 + w_1P_1 + w_2P_2 + w_3P_3 + w_4P_4 ..... + w_nP_n
\end{equation}

where $P_n$ is the Legendre polynomial of order $n$ and 
\begin{equation}
w_n = \frac{<f(x), P_n>}{<P_n, P_n>}
\end{equation}

The Legendre polynomial of the leading 6 orders are as follows:

\begin{align}
P_0(x) &= 1 \\
P_1(x) &= x \\
P_2(x) &= \frac{1}{2}(3x^2 - 1) \\
P_3(x) &= \frac{1}{2}(5x^3 - 3x) \\
P_4(x) &= \frac{1}{8}(35x^4 - 30x^2 + 3) \\
P_5(x) &= \frac{1}{8}(63x^5 - 70x^3 + 15x) \\
P_6(x) &= \frac{1}{16}(231x^6 - 315x^4 + 105x^2-5) \\
\end{align}

For this demonstration, we wish to find the fifth-order approximation of $sin(x)$. The coefficients, or in this demostration we call them "weights", $w_n$, are to be determined using gradient descent. However, since $sin(x)$ is an odd function, we can safely ignore the even order terms and only have to find the coefficients of the odd order terms.

\begin{equation}
sin(x) \approx w_1P_1(x) + w_3P_3(x) + w_5P_5(x)
\end{equation}

In [26]:
import torch
from LegendrePolynomialOfSine import ThirdOrderLegendrePolynomial, FifthOrderLegendrePolynomial

dtype = torch.float
device = torch.device("cuda")
torch.set_default_device(device)
torch.manual_seed(0)

N_SAMPLES = 4000
LEARNING_RATE = 1e-6

x = torch.linspace(-torch.pi, torch.pi, N_SAMPLES, dtype=dtype)
y = torch.sin(x)
w = torch.randn(3, device = device, dtype = dtype, requires_grad = True)

for t in range(4000):

    P3 = ThirdOrderLegendrePolynomial.apply
    P5 = FifthOrderLegendrePolynomial.apply
    
    y_pred = w[0]*x + w[1]*P3(x) + w[2]*P5(x)

    error = y_pred - y
    loss = error.pow(2).mean()

    loss.backward()

    with torch.no_grad():
        w -= w.grad * LEARNING_RATE
        w.grad[:] = 0
        
    if t % 200 == 199:
        print(f"Loss at {t+1:> 5} iteration: {loss:> 5.3f}")

Loss at   200 iteration:  11.172
Loss at   400 iteration:  10.871
Loss at   600 iteration:  10.578
Loss at   800 iteration:  10.294
Loss at  1000 iteration:  10.018
Loss at  1200 iteration:  9.750
Loss at  1400 iteration:  9.490
Loss at  1600 iteration:  9.238
Loss at  1800 iteration:  8.993
Loss at  2000 iteration:  8.755
Loss at  2200 iteration:  8.523
Loss at  2400 iteration:  8.299
Loss at  2600 iteration:  8.081
Loss at  2800 iteration:  7.870
Loss at  3000 iteration:  7.664
Loss at  3200 iteration:  7.465
Loss at  3400 iteration:  7.271
Loss at  3600 iteration:  7.083
Loss at  3800 iteration:  6.900
Loss at  4000 iteration:  6.723
